In [33]:

import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.sql._
import org.apache.log4j._
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx._
import org.apache.spark.graphx.util.GraphGenerators
import org.apache.spark.SparkContext._
import org.apache.spark.sql._
import org.apache.log4j._
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx._
import java.awt.image.BufferedImage
import java.io.File
import javax.imageio.ImageIO
import java.util.Date
import java.util.Calendar 
import java.awt.Font
import java.awt.Color;
import scala.collection.mutable.ListBuffer

import java.io.ByteArrayOutputStream;
import org.apache.hadoop.fs.{FileSystem, Path}

/** Some examples of GraphX in action with the Marvel superhero dataset! */


class Point(var x: Int = 0, var y: Int = 0) extends Serializable 
class PopulationData (val CostMatrix: Array[Array[Double]], val GeneSequences: Array[(Array[Int],Double)], var GenerationNumber:Int,var best_sequences:List[(Int,Array[Int],Int,Double)]) extends Serializable 




  val number_of_cities:Int = 20;
  val max_generations = 20;


      val r = scala.util.Random
     val cities = new Array[Point](number_of_cities);
     val index = new Array[Long](number_of_cities)
     for (i <- 0 to number_of_cities-1)
     {
       cities(i) = new Point(r.nextInt(500)+10,r.nextInt(500)+10)
       index(i) = i;
       println(cities(i).x + " " + cities(i).y);
     }
     
     val indexCities = index.zip(cities);
     val randomSequence = scala.util.Random.shuffle((0 to number_of_cities-1)).toArray
     
     
      val costmatrix = Array.ofDim[Double](number_of_cities,number_of_cities);
     //compute an euclidean distance matrix: sqrt((x2-x1)^2+(y2-y1)^2)
     for (i <-0 to number_of_cities-1)
     {
       for (j <-0 to number_of_cities-1)
       {  
         val deltax = cities(i).x-cities(j).x;
         val deltay = cities(i).y-cities(j).y;
         costmatrix (i)(j) =  math.sqrt(deltax*deltax+deltay*deltay).round;
         print(" " + costmatrix (i)(j))
       }
       println(" ")
     }
  
  //val number_of_cities:Int = 20;
  //val max_generations = 20;

0.0 252.0 427.0 380.0 495.0 451.0 508.0 321.0 416.0 380.0 514.0 434.0 590.0 40.0 529.0 262.0 263.0 410.0 184.0 323.0 
 252.0 0.0 198.0 131.0 244.0 264.0 261.0 195.0 440.0 130.0 321.0 206.0 426.0 214.0 346.0 206.0 71.0 161.0 279.0 177.0 
 427.0 198.0 0.0 159.0 180.0 346.0 216.0 351.0 619.0 96.0 378.0 244.0 497.0 394.0 409.0 388.0 247.0 159.0 477.0 327.0 
 380.0 131.0 159.0 0.0 115.0 189.0 130.0 201.0 473.0 65.0 230.0 96.0 347.0 341.0 260.0 254.0 130.0 30.0 369.0 177.0 
 495.0 244.0 180.0 115.0 0.0 211.0 36.0 284.0 549.0 134.0 221.0 114.0 337.0 456.0 253.0 349.0 242.0 85.0 475.0 261.0 
 451.0 264.0 346.0 189.0 211.0 0.0 187.0 140.0 356.0 255.0 63.0 104.0 163.0 412.0 82.0 215.0 207.0 187.0 355.0 130.0 
 508.0 261.0 216.0 130.0 36.0 187.0 0.0 275.0 533.0 161.0 190.0 99.0 303.0 468.0 221.0 344.0 249.0 101.0 475.0 253.0 
 321.0 195.0 351.0 201.0 284.0 140.0 275.0 0.0 272.0 256.0 201.0 176.0 270.0 283.0 211.0 76.0 124.0 219.0 216.0 24.0 
 416.0 440.0 619.0 473.0 549.0 356.0 533.0 272.0 0.0 52

In [34]:
  
    
    
    def to_image(vertices:Array[(Long,Point)], edges:Array[Edge[Int]],Generation:Int, Fitness:Double, PregelVertexID:Int) =
   {
      val img = new BufferedImage(550, 550, BufferedImage.TYPE_INT_RGB)
      val g = img.createGraphics();
         g.setColor(Color.WHITE)
      g.fillRect(0, 0, 550, 550)
      val myFont = new Font("Serif", Font.BOLD, 14);
      
     g.setFont(myFont)

   g.setRenderingHint(java.awt.RenderingHints.KEY_ANTIALIASING, java.awt.RenderingHints.VALUE_ANTIALIAS_ON)
  
     g.setColor(Color.BLACK)
      g.drawString("Generation:" + Generation.toString() + " Populations Vertex ID:"+ PregelVertexID +" Fitness:" + Fitness.toString(), 0, 540)
     
      for (v <- vertices)
     {
         //g.fillOval(arg0, arg1, arg2, arg3)
         g.fillOval(v._2.x-5, v._2.y-5, 10, 10);
     }
       for (e <- edges)
     {
         g.drawLine(vertices(e.srcId.toInt)._2.x, vertices(e.srcId.toInt)._2.y, vertices(e.dstId.toInt)._2.x, vertices(e.dstId.toInt)._2.y)
     }
      
    img    
  
    
   }
  

to_image: (vertices: Array[(Long, Point)], edges: Array[org.apache.spark.graphx.Edge[Int]], Generation: Int, Fitness: Double, PregelVertexID: Int)java.awt.image.BufferedImage

In [35]:

    Logger.getLogger("org").setLevel(Level.ERROR)
   //val sc = new SparkContext("local[*]", "GraphX")
   
   
   def drawsequence( isequence: Array[Int], cities: Array[(Long,Point)], file:String,Generation:Int,
                    Fitness:Double,PregelVertexId:Int):Unit =
   {
     val sequence = isequence.map(_.toLong);
     
    // val Ed:Edge[Boolean] = new Edge(1L, 2L, true);
     val EdgeArray:Array[Edge[Int]] = new Array[Edge[Int]](number_of_cities);
     
     val tspVertices = sc.makeRDD(cities); 
  
     for (i <-0 to sequence.length-2)
       EdgeArray(i) = new Edge(sequence(i),sequence(i+1),1)
     
     EdgeArray(number_of_cities-1) = new Edge(sequence(number_of_cities-1),sequence(0),1);
     
     //maybe add an edge from the last to the first node
     //val x = Array(Edge(1L, 2L, true), Edge(2L, 1L, true))
      // val edges = Array(Edge(1L,2L,1800),Edge(2L,3L,800),Edge(3L,1L,1400))
     val tspEdges = sc.makeRDD(EdgeArray); //Achtung er braucht Long!! nicht Int!
    // val tspEdges = sc.makeRDD(edges); //Achtung er braucht Long!! nicht Int!
     //val tspGraph = Graph(tspVertices, tspEdges)
     
    val img = to_image(cities,EdgeArray,Generation,Fitness,PregelVertexId)
    
    //ImageIO.write(img, "jpg", new File(file))
     
       
     val baos = new ByteArrayOutputStream();
    ImageIO.write(img, "jpg", baos);
     
       
    val pa = "/Generation_"+Generation+"_Node_"+PregelVertexId+".jpg";
    val fs = FileSystem.get(sc.hadoopConfiguration)
     val out = fs.create(new Path(pa))
    out.write(baos.toByteArray());
 out.close();      
       
       
       
   }
   

drawsequence: (isequence: Array[Int], cities: Array[(Long, Point)], file: String, Generation: Int, Fitness: Double, PregelVertexId: Int)Unit

In [36]:

      def createchild (p1:Array[Int],p2:Array[Int]):Array[Int]=
     {
         val r = scala.util.Random
      
         val p1_c = p1.clone();
         val p2_c = p2.clone();
         
         val child = new Array[Int](p1_c.length);
               val crossoverpoint = r.nextInt(p1_c.length-2);
               
               //PMX special CrossOver Method
               //ist eine art Rotation. Der zweite Eltern Teil gibt vor was ausgetauscht wird
               for (x <-0 to crossoverpoint)
               {
                  val gen = p2_c(x);
                  child(x) = gen //setze Kind bis zum crosspoint auf die stadt des Parent2
                  for (y <- (x+1) to p1_c.length-1) //durchsuche rechte seite
                  {
                     if (p1_c(y) == gen) //falls stadt schon besucht wurde links 
                       {
                         p1_c(y) = p1_c(x) //repariere sequenz 
                       }
                  }
               }
               
               //setze kind ab dem kreuzungspunkt auf die gene des parent 2
               for (y <- crossoverpoint+1 to p1_c.length-1)
                 child(y) = p1_c(y);
            //mutation:
           if (r.nextInt(100) < 1)
           {
             //println("mutate!");
             val m1 = r.nextInt(number_of_cities)
             val m2 = r.nextInt(number_of_cities)
             val gen1 = child(m1)
             val gen2 = child(m2)
             child(m1) = gen2
             child(m2) = gen1
           }
               
           return child
     }


createchild: (p1: Array[Int], p2: Array[Int])Array[Int]

In [37]:
      def compute_fitness_of_sequence (CostMatrix: Array[Array[Double]], Sequence:Array[Int]) =
      {
         var cost = 0.0;
              for (j <-0 to number_of_cities-2) //von der letzten stadt wird nicht in die Ausgangsstadt zurückgefahren.
              { 
                cost += CostMatrix(Sequence(j))(Sequence(j+1))
              }
              cost += CostMatrix(Sequence(Sequence.length-1))(Sequence(0))
         (Sequence,cost)
      }

compute_fitness_of_sequence: (CostMatrix: Array[Array[Double]], Sequence: Array[Int])(Array[Int], Double)

In [38]:

      def mergeMsg(msg1: PopulationData, msg2: PopulationData): PopulationData = {
         val x = if (msg1.best_sequences(0)._4 < msg2.best_sequences(0)._4) msg1.best_sequences else msg2.best_sequences
         val allparents = msg1.GeneSequences.union(msg2.GeneSequences)
         val ParentPopulation = new PopulationData(msg1.CostMatrix,allparents,msg1.GenerationNumber,x)
        ParentPopulation
       } 

mergeMsg: (msg1: PopulationData, msg2: PopulationData)PopulationData

In [39]:

      def vProg(v: VertexId, attr:  PopulationData, msg: PopulationData): PopulationData = { 
         val r = scala.util.Random
         val GeneSequencesofChildren = Array.ofDim[(Array[Int],Double)](5000)
             //create 5000 kids.
           for (i <-0 to 4999) 
           {
               //get random indexes
               val i1 = r.nextInt(msg.GeneSequences.length-1)
               val i2 = r.nextInt(msg.GeneSequences.length-1)
               val i3 = r.nextInt(msg.GeneSequences.length-1)
               val i4 = r.nextInt(msg.GeneSequences.length-1)
               val f1 = msg.GeneSequences(i1)._2
               val f2 = msg.GeneSequences(i2)._2
               val f3 = msg.GeneSequences(i3)._2
               val f4 = msg.GeneSequences(i4)._2              
               //select fitter (Tunierselektion mit jeweils 4 Individuen. Es werden 1000 Tuniere ausgetragen.) 
               val p1 = if (f1 < f2) msg.GeneSequences(i1) else msg.GeneSequences(i2)
               val p2 = if (f3 < f4) msg.GeneSequences(i3) else msg.GeneSequences(i4)
               
               val child = createchild(p1._1,p2._1)
               GeneSequencesofChildren(i) = (child,0)
        
           }    
           val sequence_with_fitness = GeneSequencesofChildren.map(y => {compute_fitness_of_sequence(msg.CostMatrix,y._1)})
           val evodat = new PopulationData(attr.CostMatrix,sequence_with_fitness,msg.GenerationNumber,msg.best_sequences)
        evodat; //setzt den attributwert auf dieses objekt
      } 
   

vProg: (v: org.apache.spark.graphx.VertexId, attr: PopulationData, msg: PopulationData)PopulationData

In [40]:

      def sendMsg(et: EdgeTriplet[PopulationData, Boolean]): Iterator[(VertexId, PopulationData)] = {
        val selection = et.srcAttr.GeneSequences.sortBy(x => x._2).take(1000)
        
        val fittestsequence = et.srcAttr.GeneSequences.sortBy(x => x._2).take(1)
        val fittestsequencehistory = (et.srcAttr.GenerationNumber+1,fittestsequence(0)._1,et.srcId.toInt,fittestsequence(0)._2) :: et.srcAttr.best_sequences
        val evodat = new PopulationData(et.srcAttr.CostMatrix,selection,et.srcAttr.GenerationNumber+1,fittestsequencehistory)
        Iterator((et.dstId,evodat)) //beide Klammern wichtig! (Tupel)
        //Iterator.empty;
      } 


sendMsg: (et: org.apache.spark.graphx.EdgeTriplet[PopulationData,Boolean])Iterator[(org.apache.spark.graphx.VertexId, PopulationData)]

In [41]:
   
   
     //Graph Struktur
     //compute 20 random City locations:
    val r = scala.util.Random
    
    val GeneSequences: Array[(Array[Int],Double)] = Array.ofDim[(Array[Int],Double)](5000);
    val GenSequencesFilled = GeneSequences.map(x=> {(scala.util.Random.shuffle((0 to number_of_cities-1)).toArray,0.0)})
     
    val best_sequences = List[(Int,Array[Int],Int,Double)]()
    val EvolData = new PopulationData(costmatrix,GenSequencesFilled,0,best_sequences);  //Initiale Nachricht!
 
     //Create Compute Topology Graph:
     
      val myVertices = sc.makeRDD(Array((1L, EvolData), (2L, EvolData), (3L, EvolData), (4L, EvolData), 
                                        (5L, EvolData), (6L, EvolData), (7L, EvolData), (8L, EvolData)))
     val myEdges = sc.makeRDD(Array(Edge(1L, 2L, true), Edge(2L, 1L, true), Edge(1L, 7L, true), Edge(1L, 8L, true), 
                                     Edge(2L, 5L, true), Edge(2L, 6L, true), Edge(2L, 7L, true), Edge(2L, 8L, true),
                                     Edge(3L, 5L, true), Edge(3L, 6L, true), Edge(3L, 7L, true), Edge(3L, 8L, true),
                                     Edge(4L, 5L, true), Edge(4L, 6L, true), Edge(4L, 7L, true), Edge(4L, 8L, true),
                                     
                                     Edge(5L, 1L, true), Edge(6L, 1L, true), Edge(7L, 1L, true), Edge(8L, 1L, true), 
                                     Edge(5L, 2L, true), Edge(6L, 2L, true), Edge(7L, 2L, true), Edge(8L, 2L, true),
                                     Edge(5L, 3L, true), Edge(6L, 3L, true), Edge(7L, 3L, true), Edge(8L, 3L, true),
                                     Edge(5L, 4L, true), Edge(6L, 4L, true), Edge(7L, 4L, true), Edge(8L, 4L, true)))
          
     val myGraph = Graph(myVertices, myEdges)     
  val starttime = java.time.LocalDate.now;
    val t0 = System.nanoTime()
    println("Evolution gestartet" +   starttime); 
    val resultgraph = myGraph.pregel(EvolData, max_generations, EdgeDirection.Out)(vProg, sendMsg, mergeMsg)
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0) + "ns")
    
    val endtime = java.time.LocalDate.now;
    
   println("Evolution abgeschlossen um " +  endtime  + " Duration in ms" + (t1 - t0)/1000000 ); 
    
     
    
    println("Vererbungs Stammbaum:")
     val sequencehistory =  resultgraph.vertices.map(v => v._2.best_sequences).collect();
      
     for (n <-0 to sequencehistory.length-1) //für jeden vertex
       sequencehistory(n).foreach(z => {println("Generation:" + z._1 + " Node:" + z._3 + " Sequence:" + z._2.mkString("-")+ " Fitness:" + z._4)
         
          drawsequence(z._2,indexCities,"Generation_"+z._1+"_Node_"+z._3+".jpg",z._1,z._4,z._3);
       })
   


Generation:20 Node:8 Sequence:17-3-9-2-1-16-13-0-18-8-15-7-19-5-12-14-10-11-6-4 Fitness:2240.0
Generation:19 Node:4 Sequence:17-3-9-2-1-16-13-0-18-8-15-7-19-5-12-14-10-11-6-4 Fitness:2240.0
Generation:18 Node:8 Sequence:17-3-9-2-1-16-13-0-18-8-15-7-19-5-12-14-10-11-6-4 Fitness:2240.0
Generation:17 Node:4 Sequence:17-3-9-2-1-16-13-0-18-8-15-7-19-5-14-12-10-11-6-4 Fitness:2246.0
Generation:16 Node:8 Sequence:2-9-17-3-16-1-13-0-18-8-15-7-19-5-12-14-10-11-6-4 Fitness:2267.0
Generation:15 Node:1 Sequence:17-3-2-9-1-16-13-0-18-8-15-7-19-5-14-12-10-11-6-4 Fitness:2272.0
Generation:14 Node:8 Sequence:2-9-17-3-16-1-13-0-18-8-15-7-19-5-12-14-10-11-6-4 Fitness:2267.0
Generation:13 Node:3 Sequence:2-9-17-3-16-1-13-0-18-15-19-7-8-12-14-10-5-11-6-4 Fitness:2342.0
Generation:12 Node:5 Sequence:2-9-17-3-16-1-13-0-18-7-19-15-8-12-14-10-5-11-6-4 Fitness:2380.0
Generation:11 Node:3 Sequence:9-2-4-6-16-1-13-0-18-15-7-19-8-12-14-10-5-11-17-3 Fitness:2446.0
Generation:10 Node:8 Sequence:16-1-13-0-18-8-15-7-

In [46]:
%%HTML
<img width="30%" height="30%" src="https://unispark.blob.core.windows.net/unispark-2017-04-22t15-10-29-805z/Generation_15_Node_1.jpg"/>